In [1]:
import asyncio
import os
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion

from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_message_content import ChatMessageContent

from semantic_kernel.agents.strategies.termination.default_termination_strategy import DefaultTerminationStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy

load_dotenv(override=True)

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")  
AZURE_OPENAI_API_VERSION = "2024-05-01-preview" 



In [3]:

def save_sample_documents():
    # Claim Report (Houston scenario)
    claim_report = f"""
INSURANCE CLAIM REPORT
----------------------
Claim ID: HURTX-2024-0456
Date Filed: {datetime.now().strftime('%B %d, %Y')}
Claimant: Contoso Electronics Inc.
Store Location: Contoso Electronics #TX-230
Address: 1125 Main Street, Houston, TX 77002
Contact: Gabriel Allen (Store Manager)
Contact Phone: (713) 555-4422
Contact Email: gabriel.allen@contoso.com

DAMAGE ASSESSMENT
-----------------
Incident Type: Hurricane Alicia
Date of Incident: May 15, 2024
Time of Incident: ~02:30 AM CST

DAMAGE DETAILS
--------------
- Severe roof damage and water intrusion on the south side
- Flooded inventory storage (electronics submerged)
- Electrical panels shorted due to water ingress
- Potential structural compromise in south wing support beams

Preliminary Damage Estimates:
- Inventory Loss: $380,000
- Structural Damage: $200,000
- Equipment Replacement: $160,000
- Business Interruption: TBD (further accounting needed)

Mitigation Steps Taken:
1. Secure site and prevent unauthorized entry
2. Engage emergency restoration team for water extraction
3. Move salvageable inventory to upper floors
4. Temporary roof patch installed

Photos and videos available. Awaiting professional structural engineer’s report.

FILED BY
--------
Adjuster: Lauren Chen
Fabrikam Insurance Adjuster ID: TX-ADJ-3112
Agency: Gulf Coast Claims Solutions
Initial On-Site Visit: {(datetime.now() + timedelta(days=1)).strftime('%B %d, %Y')}
""".strip()

    # Policy Details (Fabrikam Insurance)
    policy_details = f"""
FABRIKAM INSURANCE COMMERCIAL POLICY
-------------------------------------
Policy Number: CP-TX-2024087
Policyholder: Contoso Electronics Inc.
Policy Period: January 1, 2024 - December 31, 2024
Policy Type: Comprehensive Commercial Property Insurance

COVERAGE & LIMITS
-----------------
Total Insured Value: $2,000,000
Hurricane/Wind Coverage: Included
Flood Coverage: Included (Deductible: $50,000)
Business Interruption: Included (sub-limit $500,000)
Replacement Cost: Included

Sublimits:
- Building Structure: Up to $1,000,000
- Business Property (Incl. Inventory): Up to $600,000
- Electronic Equipment: Up to $200,000

DEDUCTIBLES
-----------
Hurricane/Wind Deductible: 5% of total insured value ($100,000)
Flood Deductible: $50,000
Standard Deductible: $25,000

ENDORSEMENTS & NOTES
--------------------
- Hurricane Preparedness Credit applied
- Extended Replacement Cost Endorsement active
- Business Interruption Rider active

UNDERWRITING
------------
Underwriter: Fabrikam Insurance Underwriting Dept.
Last Risk Assessment: April 5, 2024 (No major hazards noted at that time)
Annual Premium: $42,000 paid in full on January 5, 2024
    """.strip()

    # Weather Report (Hurricane Alicia in Houston)
    weather_report = f"""
NOAA SEVERE WEATHER REPORT
--------------------------
Report Date: {datetime.now().strftime('%B %d, %Y')}
Event: Hurricane Alicia (2024 Season)

STORM DETAILS
-------------
Category: 3
Max Sustained Winds: ~120 mph
Min Central Pressure: ~950 mb

HOUSTON AREA IMPACT
-------------------
Landfall: Near Galveston, TX
Time: May 15, 2024, ~02:10 AM CST
Duration of Severe Conditions: 6-8 hours over Houston

METEOROLOGY
-----------
Winds: Sustained 100-120 mph, Gusts 130+ mph
Rainfall: 10-14 inches total, peak 3 in/hr
Storm Surge: 8-10 feet along coastal areas

DAMAGE POTENTIAL
----------------
Infrastructure Risk: Very High
Property Damage Likelihood: High
Business Disruption Probability: Significant

Issued By: Dr. Marisol Reyes
National Hurricane Center - Houston Office
""".strip()

    base_directory = Path.cwd() / "insurance_docs"
    base_directory.mkdir(exist_ok=True)

    documents = {
        "claim_report.txt": claim_report,
        "policy_details.txt": policy_details,
        "weather_report.txt": weather_report,
    }

    for fname, content in documents.items():
        (base_directory / fname).write_text(content, encoding="utf-8")

    return [str(base_directory / f) for f in documents.keys()]

document_files = save_sample_documents()

print(document_files)



['c:\\Users\\juhyunlee\\workspace\\notebooks\\multiagent-sk\\multi_agent_insurance\\insurance_docs\\claim_report.txt', 'c:\\Users\\juhyunlee\\workspace\\notebooks\\multiagent-sk\\multi_agent_insurance\\insurance_docs\\policy_details.txt', 'c:\\Users\\juhyunlee\\workspace\\notebooks\\multiagent-sk\\multi_agent_insurance\\insurance_docs\\weather_report.txt']


In [4]:
def create_kernel():
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel

async def create_agents(kernel: Kernel, document_files: list[str]):
    claims_analyst = await AzureAssistantAgent.create(
        kernel=kernel,
        deployment_name=AZURE_OPENAI_DEPLOYMENT,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        name="Claims_Analyst",
        instructions="""You are an experienced Insurance Claims Analyst evaluating claim HURTX-2024-0456 for hurricane damage.

YOUR PROCESS:
1. First Response: Initial analysis only
- Review claim details and identify missing information
- DO NOT make recommendations yet

2. Final Response: Only after Policy_Verifier and Risk_Assessor input
- Consider their input for coverage and risk assessment
- Make final recommendation
- Structure in JSON format:
{
    "final_recommendation": "approve"|"deny"|"partial_approve",
    "rationale": [...],
    "approved_amount": number,
    "deductions": {...},
    "conditions": [...]
}

KEY REQUIREMENTS:
- Wait for both other agents before final recommendation
- Verify damage aligns with weather data
- Flag any inconsistencies
- Consider policy limits and deductibles""",
        enable_file_search=True,
        vector_store_filenames=document_files,
    )

    policy_verifier = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Policy_Verifier",
        instructions="""You are a Policy Verification Specialist examining claim HURTX-2024-0456.

RESPOND IMMEDIATELY AFTER Claims_Analyst's initial review with:
{
    "coverage_verification": {
        "hurricane_coverage": {...},
        "flood_coverage": {...},
        "business_interruption": {...}
    },
    "applicable_deductibles": [...],
    "available_limits": {...},
    "coverage_gaps": [...],
    "policy_compliance": "compliant"|"non_compliant",
    "special_conditions": [...]
}""",
    )

    risk_assessor = ChatCompletionAgent(
        service_id=AZURE_OPENAI_DEPLOYMENT,
        kernel=kernel,
        name="Risk_Assessor",
        instructions="""You are a Risk Assessment Specialist analyzing claim HURTX-2024-0456.

RESPOND AFTER Policy_Verifier with:
{
    "risk_analysis": {
        "weather_correlation": {...},
        "mitigation_effectiveness": {...},
        "future_risks": [...]
    },
    "damage_validation": {...},
    "mitigation_assessment": {...},
    "recommendations": [...],
    "risk_score": number
}""",
    )

    return claims_analyst, policy_verifier, risk_assessor

# creating custom termination strategy
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "final_recommendation" in history[-1].content.lower()



In [5]:

kernel = create_kernel()
claims_analyst, policy_verifier, risk_assessor = await create_agents(kernel, document_files)

try:
    chat = AgentGroupChat(
        agents=[claims_analyst, policy_verifier, risk_assessor],
        # termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
        termination_strategy=ApprovalTerminationStrategy(agents=[claims_analyst], maximum_iterations=10),
    )

    claim_id = "HURTX-2024-0456"
    input = f"Please analyze claim {claim_id} from the provided documents. Each agent should follow their specific instructions and respond in turn."
    # Add the initial message
    await chat.add_chat_message(
        ChatMessageContent(
            role=AuthorRole.USER, 
            content=f"Please analyze claim {claim_id} from the provided documents. Each agent should follow their specific instructions and respond in turn."
        )
    )
    
    
    # print green user icon and the input
    print("\n 🙂 User:")
    print(input)


    current_agent = None
    async for content in chat.invoke():
        # print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
        if content.name != current_agent:
            current_agent = content.name
            print(f"\n👤 {current_agent}:")

        if content.content:
            print(content.content.strip())
# print(f"# IS COMPLETE: {chat.is_complete}")
except Exception as e:
    await chat.reset()


 🙂 User:
Please analyze claim HURTX-2024-0456 from the provided documents. Each agent should follow their specific instructions and respond in turn.

👤 Claims_Analyst:
**Initial Analysis of Claim HURTX-2024-0456:**

1. **Claim Details:**
   - Claim ID: HURTX-2024-0456
   - Date Filed: January 21, 2025
   - Claimant: Contoso Electronics Inc.
   - Store Location: Houston, TX
   - Incident Type: Hurricane Alicia
   - Date of Incident: May 15, 2024

2. **Damage Assessment:**
   - **Severe roof damage**
   - **Water intrusion** on the south side of the building
   - **Flooded inventory** storage causing damage to electronics
   - **Electrical panels** shorted due to water ingress
   - **Potential structural compromise** in the support beams

3. **Preliminary Damage Estimates:**
   - Inventory Loss: $380,000
   - Structural Damage: $200,000
   - Equipment Replacement: $160,000
   - Business Interruption: To be determined

4. **Mitigation Steps Taken:**
   - Secured the site
   - Engaged eme